In [149]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import codecs
import os
import gzip
import sys
from bs4 import BeautifulSoup
from itertools import chain        
from contextlib import ExitStack 
import time
import json
from sklearn.metrics import silhouette_samples
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import pymorphy2
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
from multiprocessing.dummy import Lock
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from pytorch_transformers import BertTokenizer, BertConfig
from transformers import  BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [390]:
morph = pymorphy2.MorphAnalyzer()

In [391]:
stemmerR = SnowballStemmer('russian')
stemmerE = SnowballStemmer('english')

In [392]:
def inp_for_titles(line):
    data = line.split('\t', 1)
    doc_id = int(data[0])
    if len(data[1]) == 1:
        title = ''
    else:
        title = data[1]
    doc_to_title[doc_id] = title[:-1]

In [393]:
doc_to_title = {}
with codecs.open('docs_titles.tsv', 'r', 'utf8') as f:
    next(f)
    with ThreadPool(10) as pool:
        pool.map(inp_for_titles, f)
print(len(doc_to_title))

28026


In [394]:
doc_to_title

{15731: 'ВАЗ 21213 | Замена подшипников ступицы | Нива',
 14829: 'Ваз 2107 оптом в Сочи. Сравнить цены, купить потребительские товары на Tiu.ru',
 15764: 'Купить ступица Лада калина2. Трансмиссия - переходные ступицы цена, замена, тюнинг.',
 17669: 'Классика 21010 - 21074',
 14852: 'Ступица Нива — замена подшипника своими руками',
 15458: 'ВАЗ 2110',
 14899: 'Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123',
 16879: 'Купить Подшипники и ступицы FAG (Страница 23)',
 16310: 'HorsePowers — автомобильный интернет портал » Отзыв владельца ВАЗ 2121 Нива 2007 года',
 15440: 'Новости и сообщения из официальной группы Вконтакте торговой компании 33 Sport - Магазины - Тольятти',
 16242: 'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru',
 16383: 'Ступицы - OLX.ua - страница 80',
 15580: 'маааленькая проблемка — бортжурнал Автокам 2160 ╬ 1994 года на DRIVE2',
 16840: 'Разгруженные полуоси для Нива (24 шлица 765 мм)',
 17519: 'Прошивки для нива м7.9.7 скачать - Фа

In [395]:
stop_words = set(stopwords.words(["russian", "english"]))

In [396]:
lock = Lock()

In [398]:
def Parser_for_titles(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [stemmerR.stem(stemmerE.stem(word)) for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [399]:
def Parser_for_titles_without_stemming(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [word for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [400]:
def Parser_for_text(ID):
    with lock:
        text_id = df[df.id == ID].text.values
    text_id = text_id[0]
    stemmed_text = [stemmerR.stem(stemmerE.stem(word)) for word in text_id.split(' ')[:512]]
    with lock:
        vector_of_text[str(ID)] = ' '.join([word for word in stemmed_text])

In [120]:
df_iter = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/data/project.csv', encoding='utf-8', chunksize=500)

In [159]:
i = 0
for df in df_iter:
    df.fillna(' ')
    vector_of_text = {}
    idx = df.id.values
    with ThreadPool(10) as pool:
        pool.map(Parser_for_text, idx)
    with open('texts/text_stemmed_part_{:05d}.json'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)
        record = json.dumps(vector_of_text, ensure_ascii=False)
        print(record, file=f_json)
    i += 1

есть 57 файлов с индексами страниц по 500

In [401]:
vec = {}
with ThreadPool(10) as pool:
    pool.map(Parser_for_titles_without_stemming, list(range(1,28027)))

In [402]:
vec

{1: 'м б аншина центр репродукции и генетики фертимед г москва',
 2: 'переводы киви кошелька',
 3: 'проект патрули времени реабилитация духовных существ просмотр темы деньги и экономика обмен и финансы деньги без процентов',
 4: 'блог клуб преподавание в начальных классах портал профессионального образования чеченской республики',
 5: 'как быстро понизить холестерин высокий холестерин симптомы и лечение народными средствами в домашних условиях',
 6: 'как составить резюме',
 7: 'изюм',
 8: 'проект ладушки ладушки для детей раннего возраста',
 9: 'сколько калорий в дыне торпеда колхозница сколько калорий в 1 кг дыни в 100 гр в сушеной',
 10: 'нужна помощь доска объявлений заработать деньги денежная помощь кредитная помощь',
 11: 'как подкормить огурцы дрожжами во саду и в огороде',
 12: 'прозрачные воды южных морей 2',
 13: 'вопросы по продолжению выплат и набора опс в 2014г агентская работа форум pensiamarket ru навигатор пенсионного рынка',
 14: 'экстернат по новому закону об образован

In [403]:
with open('titles_without_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

In [27]:
with open('titles_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

Застемил заголовки - уже прекрасно!

## Мутим фичи

In [1]:
!pip install fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-macosx_10_12_x86_64.whl size=324287 sha256=1aca536a5f62e01a9433b11746a610136aa323a703f84c1f45af47de11095c96
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
!pip install gensim

     |████████████████████████████████| 24.2 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 117 kB 2.2 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.0.1-py3-none-any.whl size=108249 sha256=54b93ee534545cb3657fe4c27116c6ea53bd50ab3c37028568418f11c50e2d07
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/34/3d/14/f19c01a19c9201cdb6a76b049904d5226912569be919ad1eae
Successfully built smart-open
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.fasttext import load_facebook_model

Взяли заголовки

In [3]:
vec = {}

In [4]:
with open('titles_stemming.json', 'r', encoding='utf8') as file:
    vec = json.load(file)

In [5]:
sentences = []
for value in vec.values():
    sentences.append(value)

In [6]:
model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')

In [7]:
len(sentences)

28026

In [8]:
first = sentences[0]
first

'м б аншин центр репродукц и генетик фертимед г москв'

In [9]:
len(model_fasttext.wv[first])

100

Давайте попробуем сделать общий файл с эмеддингами текстов.. не просто так же их парсили

Делаем эмбединги и запоминаем их

In [40]:
filenames = os.listdir('texts_57_json')
texts_embeddings = {}
texts_embeding_to_json = {}
for f_name in filenames:
    file_n = 'texts_57_json/' + f_name
    with open(file_n, mode='r') as file:
        d = json.load(file)  # все тексты 
        for key, val in d.items():
            embedding = model_fasttext.wv[val]
            embedding_string = tuple(map(str, embedding))
            texts_embeddings[key] = embedding
            texts_embeding_to_json[key] = embedding_string

In [41]:
texts_embeddings['66']

array([ 2.09348174e-04, -2.75553117e-04,  8.44116003e-05,  7.03980681e-04,
        8.31300131e-05, -4.98171430e-05, -1.72995031e-04,  3.87511274e-04,
        4.15297778e-04,  3.81176564e-04, -6.21469662e-05, -4.27063758e-04,
       -4.51792555e-04,  1.69587351e-04,  1.74802059e-04, -3.35838093e-04,
        1.27144460e-06, -1.62012526e-04,  4.94475753e-05, -2.60678207e-04,
        3.67626817e-05, -6.08271221e-04,  5.91311138e-04,  3.38482962e-04,
       -2.85922346e-04, -2.80239765e-04, -7.78300000e-06, -5.78220759e-04,
       -4.90745471e-04,  2.27174954e-04, -5.47075411e-04, -9.38452024e-04,
        2.81078101e-04,  4.84719640e-04,  4.69160703e-04,  2.44825671e-04,
       -4.23335208e-04,  6.67298213e-04, -2.71135184e-04,  3.54419491e-04,
        5.89366537e-04, -4.00906225e-04,  2.00902432e-04, -3.34325450e-04,
        2.51527323e-04,  4.58418544e-05,  2.72824720e-04, -1.37650481e-04,
       -2.49142831e-05, -6.46616754e-05,  3.63226020e-04, -1.57629067e-04,
       -5.82841516e-04, -

In [42]:
texts_embeding_to_json['66']

('0.00020934817',
 '-0.00027555312',
 '8.44116e-05',
 '0.0007039807',
 '8.313001e-05',
 '-4.9817143e-05',
 '-0.00017299503',
 '0.00038751127',
 '0.00041529778',
 '0.00038117656',
 '-6.2146966e-05',
 '-0.00042706376',
 '-0.00045179256',
 '0.00016958735',
 '0.00017480206',
 '-0.0003358381',
 '1.2714446e-06',
 '-0.00016201253',
 '4.9447575e-05',
 '-0.0002606782',
 '3.676268e-05',
 '-0.0006082712',
 '0.00059131114',
 '0.00033848296',
 '-0.00028592235',
 '-0.00028023976',
 '-7.783e-06',
 '-0.00057822076',
 '-0.0004907455',
 '0.00022717495',
 '-0.0005470754',
 '-0.000938452',
 '0.0002810781',
 '0.00048471964',
 '0.0004691607',
 '0.00024482567',
 '-0.0004233352',
 '0.0006672982',
 '-0.00027113518',
 '0.0003544195',
 '0.00058936654',
 '-0.00040090622',
 '0.00020090243',
 '-0.00033432545',
 '0.00025152732',
 '4.5841854e-05',
 '0.00027282472',
 '-0.00013765048',
 '-2.4914283e-05',
 '-6.4661675e-05',
 '0.00036322602',
 '-0.00015762907',
 '-0.0005828415',
 '-0.00036128244',
 '0.0001296946',
 '0.00

Запись в файл стринговых эмбеддингов

In [43]:
with open('texts_embeddings.json', 'w', encoding='utf8') as file:
    json.dump(texts_embeding_to_json, file)

Попробуем прочитать эмбеддинги - не забудь перевести в float

In [54]:
with open('texts_embeddings.json', 'r', encoding='utf8') as file:
    texts_embedding_from_json = json.load(file)

In [55]:
for key, val in texts_embedding_from_json.items():
    embedding = np.array(list(map(float, texts_embedding_from_json[key])))
    texts_embedding_from_json[key] = embedding

In [56]:
texts_embedding_from_json

{'66': array([ 2.09348170e-04, -2.75553120e-04,  8.44116000e-05,  7.03980700e-04,
         8.31300100e-05, -4.98171430e-05, -1.72995030e-04,  3.87511270e-04,
         4.15297780e-04,  3.81176560e-04, -6.21469660e-05, -4.27063760e-04,
        -4.51792560e-04,  1.69587350e-04,  1.74802060e-04, -3.35838100e-04,
         1.27144460e-06, -1.62012530e-04,  4.94475750e-05, -2.60678200e-04,
         3.67626800e-05, -6.08271200e-04,  5.91311140e-04,  3.38482960e-04,
        -2.85922350e-04, -2.80239760e-04, -7.78300000e-06, -5.78220760e-04,
        -4.90745500e-04,  2.27174950e-04, -5.47075400e-04, -9.38452000e-04,
         2.81078100e-04,  4.84719640e-04,  4.69160700e-04,  2.44825670e-04,
        -4.23335200e-04,  6.67298200e-04, -2.71135180e-04,  3.54419500e-04,
         5.89366540e-04, -4.00906220e-04,  2.00902430e-04, -3.34325450e-04,
         2.51527320e-04,  4.58418540e-05,  2.72824720e-04, -1.37650480e-04,
        -2.49142830e-05, -6.46616750e-05,  3.63226020e-04, -1.57629070e-04,
      

## Фичи

Если фасттекст применять к текстам после стемминга, результат получается лучше почему-то.

Для заголовков:
1) Метки кластеров внутри группы
2) Силуэт для объекта группы

In [204]:
class Import:
    def __init__(self):
        self.doc_to_title = {}
        #self.model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')
        with open('/Users/egor/Desktop/Техносфера/MachineLearning/project/docs_titles.tsv') as f:
            for num_line, line in enumerate(f):
                if num_line == 0:
                    continue
                data = line.strip().split('\t', 1)
                doc_id = int(data[0])
                if len(data) == 1:
                    title = ''
                else:
                    title = data[1]
                self.doc_to_title[doc_id] = title
        self.doc_to_vec = TfidfVectorizer().fit_transform([self.doc_to_title[i] for i in self.doc_to_title.keys()])
        self.top_k_cosine = 5 # k ближайших косинусных расстояний
        self.n_features_jac = 3
        self.extra_features = self.cosine
   
    def cosine(self, group):
        
        def jaccard(group):
            X = np.empty(shape=(group.size, self.n_features_jac), dtype=np.float)
            for i, doc_ID in enumerate(group):
                dist = np.empty(shape=(group.size, ), dtype=np.float)
                vec = set(self.doc_to_title[doc_ID])
                for j, doc_j in enumerate(group):
                    if doc_ID == doc_j:
                        dist[j] = 0
                    else:
                        vec_j = set(self.doc_to_title[doc_j])
                        intersec = len(vec.intersection(vec_j))
                        if intersec > 0:
                            dist[j] = intersec/(len(vec) + len(vec_j) - intersec)
                        else:
                            dist[j] = 0
                X[i] = np.sort(dist)[-self.n_features_jac:]
            return X
    
        n = (self.n_features)//2
        X = np.empty(shape=(group.size, self.n_features), dtype=np.float)
        for i, title in enumerate(pairwise_distances(self.doc_to_vec[group], metric='cosine')):
            X[i, :n] = sorted(title)[1:n + 1]
        X[:, n:] = np.mean(X[:, :n], axis=0)
        X[:, :n] /= X[:, n:]
        if (self.n_features_jac==0):
            return X
        X_jac = jaccard(group)
        return np.hstack((X,X_jac))
                
    
    def make_train(self):
        train_data = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/train_groups.csv')
        traingroups_titledata = {}
        for i in range(len(train_data)):
            new_doc = train_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            target = new_doc['target']
            title = self.doc_to_title[doc_id]
            if doc_group not in traingroups_titledata:
                traingroups_titledata[doc_group] = []
            traingroups_titledata[doc_group].append((doc_id, title, target))
        
        y_train = []
        X_train = []
        groups_train = []
        for new_group in traingroups_titledata:
            docs = traingroups_titledata[new_group]
            for k, (doc_id, title, target_id) in enumerate(docs):
                y_train.append(target_id)
                groups_train.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j, target_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_train.append(sorted(all_dist, reverse=True)[0:25]    )
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        groups_train = np.array(groups_train)
        
        # Класстеризация внутри группы:
        i = 0
        My_X = np.zeros((train_data.shape[0], self.top_k_cosine + 2)) # кол-во новых фичей
        groups = train_data.groupby('group_id')
        rand_index = []
        for group_id, group_indx in groups.groups.items():
            group = train_data.iloc[group_indx]
            docs_id = group.doc_id.values
            # сначала заберем все файлы из группы
            embedded_texts = []
            group_titles = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embeddings[str(idx)])
            embedded_titles = model_fasttext.wv[group_titles]
            targets = group.target.values
            model_titles = DBSCAN(0.475, metric="cosine", min_samples=7)
            labels_titles = model_titles.fit_predict(embedded_titles)
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.7, metric="cosine", min_samples=5)
            labels_texts = model_texts.fit_predict(embedded_texts)
            # добавим фичи из расстояний
            # получим матрицу из попарных расстояний
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine + 1]
            My_X[i: i + group_indx.size, :self.top_k_cosine] = cos_features
            My_X[i: i + group_indx.size, -2] = labels_titles
            My_X[i: i + group_indx.size, -1] = labels_texts
            #My_X[i: i + group_indx.size, -2] = silhouette
            #My_X[i: i + group_indx.size, :-2] = self.extra_features(group.doc_id)
            i += group_indx.size
        
        
        X_train = np.concatenate((X_train, My_X), axis=1)
        return X_train, y_train, groups_train
    
    def make_test(self):
        test_data = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/test_groups.csv')
        testgroups_titledata = {}
        for i in range(len(test_data)):
            new_doc = test_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            title = self.doc_to_title[doc_id]
            if doc_group not in testgroups_titledata:
                testgroups_titledata[doc_group] = []
            testgroups_titledata[doc_group].append((doc_id, title))
            
        X_test = []
        groups_test = []
        for new_group in testgroups_titledata:
            docs = testgroups_titledata[new_group]
            for k, (doc_id, title) in enumerate(docs):
                groups_test.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_test.append(sorted(all_dist, reverse=True)[0:25])
        X_test = np.array(X_test)
        
        i = 0
        My_X = np.zeros((test_data.shape[0], self.top_k_cosine + 2)) # кол-во новых фичей
        groups = test_data.groupby('group_id')
        for group_id, group_indx in groups.groups.items():
            group = test_data.iloc[group_indx]
            docs_id = group.doc_id.values
            group_titles = []
            embedded_texts = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embeddings[str(idx)])
            embedded_titles = model_fasttext.wv[group_titles]
            model_titles = DBSCAN(0.475, metric="cosine", min_samples=7)
            labels_titles = model_titles.fit_predict(embedded_titles)
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.7, metric="cosine", min_samples=5)
            labels_texts = model_texts.fit_predict(embedded_texts)
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine + 1]
            My_X[i: i + group_indx.size, :self.top_k_cosine] = cos_features
            My_X[i: i + group_indx.size, -2] = labels_titles
            My_X[i: i + group_indx.size, -1] = labels_texts
            My_X[i: i + group_indx.size, -2] = labels_titles
            My_X[i: i + group_indx.size, -1] = labels_texts
            #My_X[i: i + group_indx.size, -2] = silhouette
            #My_X[i: i + group_indx.size, :-2] = self.extra_features(group.doc_id)
            i += group_indx.size
        
        X_test = np.concatenate((X_test, My_X), axis=1)
        groups_test = np.array(groups_test)
        return X_test, groups_test, test_data['pair_id'].tolist()

In [205]:
importer = Import()

In [206]:
X_train, y_train, groups_train = importer.make_train()

In [207]:
X_test, groups_test, pair_ids = importer.make_test()

## Модельки

In [170]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator
from statistics import mean
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neural_network import MLPClassifier
import json
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import catboost
from sklearn.svm import SVC

In [208]:
my_sk = StandardScaler()
X_train_sk = my_sk.fit_transform(X_train)
X_test_sk = my_sk.fit_transform(X_test)

In [209]:
class Voiting(BaseEstimator):
    def __init__(self):
        self.clf_e_1 = RandomForestClassifier(max_depth=10)
        self.clf_e_2 = ExtraTreesClassifier(max_depth=15)
        self.clf_e_3 = LogisticRegression()
        self.clf_e_4 = catboost.CatBoostClassifier()
        self.clf_e_5 = MLPClassifier(max_iter=800, hidden_layer_sizes=(10, 10), random_state=42)
        self.clf_e_6 = KNeighborsClassifier(10)

    def fit(self, X, y):
        self.clf_e_1.fit(X, y)
        self.clf_e_2.fit(X, y)
        self.clf_e_3.fit(X, y)
        self.clf_e_4.fit(X, y)
        self.clf_e_5.fit(X, y)
        self.clf_e_6.fit(X, y)

    def predict(self, X):
        self.pred = np.transpose(np.array([self.clf_e_1.predict(X),
                                self.clf_e_2.predict(X),
                                self.clf_e_3.predict(X),
                                self.clf_e_4.predict(X),
                                self.clf_e_5.predict(X),
                                self.clf_e_6.predict(X)]))
        def pr(line):
            ar, count = np.unique(line, return_counts = True)
            return ar[np.argmax(count)]
        predict = np.array([pr(line) for line in self.pred])
        return predict   

In [210]:
clf_e_1 = RandomForestClassifier(max_depth=10, n_estimators=150)
clf_e_2 = ExtraTreesClassifier(max_depth=15)
clf_e_3 = LogisticRegression()
clf_e_4 = catboost.CatBoostClassifier()
#clf_e_5 = MLPClassifier(max_iter=800, hidden_layer_sizes=(10, 10), random_state=42)

In [211]:
clf_e_1.fit(X_train_sk, y_train)
clf_e_2.fit(X_train_sk, y_train)
clf_e_3.fit(X_train_sk, y_train)
clf_e_4.fit(X_train_sk, y_train)
#clf_e_5.fit(X_train_sk, y_train)

Learning rate set to 0.029437
0:	learn: 0.6707629	total: 23ms	remaining: 23s
1:	learn: 0.6507075	total: 39.6ms	remaining: 19.7s
2:	learn: 0.6311907	total: 54.2ms	remaining: 18s
3:	learn: 0.6142750	total: 67.2ms	remaining: 16.7s
4:	learn: 0.5983962	total: 79.1ms	remaining: 15.7s
5:	learn: 0.5835530	total: 92.6ms	remaining: 15.3s
6:	learn: 0.5708696	total: 105ms	remaining: 14.9s
7:	learn: 0.5575853	total: 117ms	remaining: 14.5s
8:	learn: 0.5454985	total: 130ms	remaining: 14.3s
9:	learn: 0.5351978	total: 141ms	remaining: 14s
10:	learn: 0.5242587	total: 154ms	remaining: 13.8s
11:	learn: 0.5143486	total: 167ms	remaining: 13.7s
12:	learn: 0.5067457	total: 178ms	remaining: 13.5s
13:	learn: 0.4987405	total: 191ms	remaining: 13.5s
14:	learn: 0.4904731	total: 207ms	remaining: 13.6s
15:	learn: 0.4829472	total: 225ms	remaining: 13.8s
16:	learn: 0.4762364	total: 238ms	remaining: 13.8s
17:	learn: 0.4699628	total: 252ms	remaining: 13.7s
18:	learn: 0.4639826	total: 269ms	remaining: 13.9s
19:	learn: 0.

168:	learn: 0.3567292	total: 2.71s	remaining: 13.3s
169:	learn: 0.3566354	total: 2.73s	remaining: 13.3s
170:	learn: 0.3565377	total: 2.74s	remaining: 13.3s
171:	learn: 0.3565065	total: 2.75s	remaining: 13.3s
172:	learn: 0.3564149	total: 2.77s	remaining: 13.2s
173:	learn: 0.3562996	total: 2.78s	remaining: 13.2s
174:	learn: 0.3561865	total: 2.79s	remaining: 13.2s
175:	learn: 0.3561315	total: 2.81s	remaining: 13.2s
176:	learn: 0.3560817	total: 2.83s	remaining: 13.1s
177:	learn: 0.3559833	total: 2.84s	remaining: 13.1s
178:	learn: 0.3559502	total: 2.85s	remaining: 13.1s
179:	learn: 0.3558379	total: 2.86s	remaining: 13s
180:	learn: 0.3557674	total: 2.87s	remaining: 13s
181:	learn: 0.3556438	total: 2.89s	remaining: 13s
182:	learn: 0.3555908	total: 2.9s	remaining: 12.9s
183:	learn: 0.3554534	total: 2.91s	remaining: 12.9s
184:	learn: 0.3554205	total: 2.93s	remaining: 12.9s
185:	learn: 0.3553349	total: 2.94s	remaining: 12.9s
186:	learn: 0.3552808	total: 2.96s	remaining: 12.9s
187:	learn: 0.35518

339:	learn: 0.3432564	total: 5.22s	remaining: 10.1s
340:	learn: 0.3431797	total: 5.23s	remaining: 10.1s
341:	learn: 0.3431095	total: 5.25s	remaining: 10.1s
342:	learn: 0.3430014	total: 5.26s	remaining: 10.1s
343:	learn: 0.3429057	total: 5.28s	remaining: 10.1s
344:	learn: 0.3428155	total: 5.3s	remaining: 10.1s
345:	learn: 0.3426398	total: 5.31s	remaining: 10s
346:	learn: 0.3425225	total: 5.33s	remaining: 10s
347:	learn: 0.3424186	total: 5.34s	remaining: 10s
348:	learn: 0.3423075	total: 5.35s	remaining: 9.98s
349:	learn: 0.3422253	total: 5.37s	remaining: 9.96s
350:	learn: 0.3421552	total: 5.38s	remaining: 9.94s
351:	learn: 0.3420813	total: 5.39s	remaining: 9.92s
352:	learn: 0.3420732	total: 5.4s	remaining: 9.9s
353:	learn: 0.3419275	total: 5.41s	remaining: 9.88s
354:	learn: 0.3418315	total: 5.43s	remaining: 9.86s
355:	learn: 0.3416948	total: 5.44s	remaining: 9.85s
356:	learn: 0.3416513	total: 5.46s	remaining: 9.83s
357:	learn: 0.3415414	total: 5.47s	remaining: 9.81s
358:	learn: 0.3414633

508:	learn: 0.3274848	total: 7.74s	remaining: 7.46s
509:	learn: 0.3274196	total: 7.75s	remaining: 7.45s
510:	learn: 0.3273062	total: 7.77s	remaining: 7.43s
511:	learn: 0.3272872	total: 7.78s	remaining: 7.42s
512:	learn: 0.3272798	total: 7.8s	remaining: 7.4s
513:	learn: 0.3271657	total: 7.82s	remaining: 7.39s
514:	learn: 0.3270910	total: 7.83s	remaining: 7.37s
515:	learn: 0.3270336	total: 7.84s	remaining: 7.36s
516:	learn: 0.3269479	total: 7.86s	remaining: 7.34s
517:	learn: 0.3268425	total: 7.87s	remaining: 7.32s
518:	learn: 0.3267736	total: 7.88s	remaining: 7.3s
519:	learn: 0.3266752	total: 7.89s	remaining: 7.29s
520:	learn: 0.3265891	total: 7.91s	remaining: 7.27s
521:	learn: 0.3265136	total: 7.92s	remaining: 7.25s
522:	learn: 0.3264417	total: 7.93s	remaining: 7.23s
523:	learn: 0.3263623	total: 7.95s	remaining: 7.22s
524:	learn: 0.3262652	total: 7.96s	remaining: 7.21s
525:	learn: 0.3262081	total: 7.98s	remaining: 7.19s
526:	learn: 0.3261141	total: 7.99s	remaining: 7.17s
527:	learn: 0.3

676:	learn: 0.3137822	total: 10.2s	remaining: 4.88s
677:	learn: 0.3137368	total: 10.3s	remaining: 4.87s
678:	learn: 0.3137157	total: 10.3s	remaining: 4.85s
679:	learn: 0.3136184	total: 10.3s	remaining: 4.84s
680:	learn: 0.3135058	total: 10.3s	remaining: 4.82s
681:	learn: 0.3134188	total: 10.3s	remaining: 4.81s
682:	learn: 0.3134158	total: 10.3s	remaining: 4.79s
683:	learn: 0.3132701	total: 10.3s	remaining: 4.78s
684:	learn: 0.3132156	total: 10.4s	remaining: 4.76s
685:	learn: 0.3131264	total: 10.4s	remaining: 4.75s
686:	learn: 0.3130066	total: 10.4s	remaining: 4.73s
687:	learn: 0.3129183	total: 10.4s	remaining: 4.71s
688:	learn: 0.3128901	total: 10.4s	remaining: 4.69s
689:	learn: 0.3127432	total: 10.4s	remaining: 4.68s
690:	learn: 0.3127212	total: 10.4s	remaining: 4.66s
691:	learn: 0.3126635	total: 10.4s	remaining: 4.65s
692:	learn: 0.3125778	total: 10.5s	remaining: 4.63s
693:	learn: 0.3124746	total: 10.5s	remaining: 4.62s
694:	learn: 0.3123980	total: 10.5s	remaining: 4.6s
695:	learn: 0

844:	learn: 0.3013035	total: 12.7s	remaining: 2.33s
845:	learn: 0.3012027	total: 12.7s	remaining: 2.31s
846:	learn: 0.3011635	total: 12.7s	remaining: 2.3s
847:	learn: 0.3010997	total: 12.7s	remaining: 2.28s
848:	learn: 0.3009825	total: 12.8s	remaining: 2.27s
849:	learn: 0.3009475	total: 12.8s	remaining: 2.25s
850:	learn: 0.3008717	total: 12.8s	remaining: 2.24s
851:	learn: 0.3008123	total: 12.8s	remaining: 2.22s
852:	learn: 0.3007659	total: 12.8s	remaining: 2.21s
853:	learn: 0.3006816	total: 12.8s	remaining: 2.19s
854:	learn: 0.3005583	total: 12.8s	remaining: 2.18s
855:	learn: 0.3004597	total: 12.9s	remaining: 2.16s
856:	learn: 0.3003530	total: 12.9s	remaining: 2.15s
857:	learn: 0.3002603	total: 12.9s	remaining: 2.13s
858:	learn: 0.3001800	total: 12.9s	remaining: 2.12s
859:	learn: 0.3000773	total: 12.9s	remaining: 2.1s
860:	learn: 0.3000185	total: 12.9s	remaining: 2.09s
861:	learn: 0.2998843	total: 12.9s	remaining: 2.07s
862:	learn: 0.2997744	total: 13s	remaining: 2.06s
863:	learn: 0.29

In [212]:
res = (clf_e_1.predict_proba(X_train_sk)[:, 1] + clf_e_2.predict_proba(X_train_sk)[: ,1] + clf_e_3.predict_proba(X_train_sk)[:, 1]
+ clf_e_4.predict_proba(X_train_sk)[:, 1])/ 4

In [213]:
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res > th, dtype=int)))

0.6784003332639034
0.7523867069486404
0.7952490214603859
0.804190644828095
0.7881028938906752
0.747719298245614
0.6799007444168734


In [177]:
f1_score(y_train, np.array(res > 0.4, dtype=int))

0.8037825059101655

In [178]:
test = (clf_e_1.predict_proba(X_test_sk)[:, 1] + clf_e_2.predict_proba(X_test_sk)[: ,1] + clf_e_3.predict_proba(X_test_sk)[:, 1]
+ clf_e_4.predict_proba(X_test_sk)[:, 1])/ 4

In [179]:
y_test = np.array(test > 0.4, dtype=int)

In [180]:
with open("/Users/egor/Desktop/resultBest.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_test)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_test[i]) + str("\n"))